### Load libs

In [84]:
import pandas as pd
import numpy as np
import sklearn
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
import re
from scipy import spatial
import time
import multiprocessing

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load data

In [85]:
df1=pd.read_excel(r"datasets/extremism.xlsx")

df=pd.DataFrame()
df['id'] = df1['id']
df['text'] = df1['text']
df['class'] = df1['class']
df = df.dropna()

In [86]:
df

,id,text,class
0,1,Таким образом никто не внес большего вклад...,3.0
1,2,"Можно с уверенностью сказать, что все полезное...",3.0
2,3,Славяне должны на нас работать. Если они нам б...,3.0
3,4,"Русское государство росло, развивалось из свои...",3.0
4,5,Поставим власть под контроль народа,3.0
...,...,...,...
304,305,Социальное. Соответствующий подход предполагае...,0.0
305,306,Рост продуктивности человеческой деятельности ...,0.0
306,307,"Социальная дифференциация, в том числе по этни...",0.0
307,308,Демографический рост и расширение экономическо...,0.0


### Load models

In [88]:
w2v_model = Word2Vec(
    min_count=10,
    window=10,
    size=300,
    negative=10,
    alpha=0.5,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1,workers=4)

exp_model = w2v_model.wv.load_word2vec_format(r'models/exp_model.bin', binary=True)

In [89]:
exp_model.most_similar('кошка')

KeyError: "word 'кошка' not in vocabulary"

## Prepare funcs

### Tokenization

In [90]:
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

def comma2space(word):
    new_word = ''
    try:
        for char in word:
            if char in patterns:
                char = ' '
            new_word += char
    except TypeError:
        pass
    return new_word
    
def lemmatize(doc):
    doc = re.sub(patterns, '', doc)
    tokens = []
    for token in doc.split():
        if token:
            
            token = token.lower()
            token = token.strip()
            tokens.append(token)
    return tokens

def dropstopwords(docs):
    stopwords = nltk.corpus.stopwords.words('russian')
    sen =  []
    for doc in docs:
        if (doc not in stopwords):
            sen.append(doc)
    return sen

def dropcomma(doc):
    new_sen = ''
    for char in doc:
        if (char not in patterns):
            new_sen += char
    return new_sen

def tokenize(docs):
    docs = comma2space(docs)
    docs = lemmatize(docs)
    docs = dropstopwords(docs)
    return docs

### Medium word-vector

In [163]:
def mediumVector(sen):
    medVec = np.zeros(300).reshape((1, 300))
    count = 0
    for word in sen:
        try:
            vec = exp_model.word_vec(word)
            medVec += vec
            count += 1
        except:
            pass
    if count >= 0 :
        medVec = medVec / count
    return medVec

## Organise data 

In [164]:
data=df['text'].apply(tokenize)
data

0      [таким, образом, никто, внес, большего, вклада...
1      [уверенностью, сказать, полезное, хорошее, сов...
2      [славяне, должны, работать, нам, нужны, могут,...
3      [русское, государство, росло, развивалось, сво...
4                   [поставим, власть, контроль, народа]
                             ...                        
304    [социальное, соответствующий, подход, предпола...
305    [рост, продуктивности, человеческой, деятельно...
306    [социальная, дифференциация, числе, этническом...
307    [демографический, рост, расширение, экономичес...
308    [соответственно, политика, возникает, связи, у...
Name: text, Length: 309, dtype: object

In [165]:
df['tokenized']=df['text'].apply(tokenize)
df['medVec']=df['tokenized'].apply(mediumVector)
df

,id,text,class,tokenized,medVec
0,1,Таким образом никто не внес большего вклад...,3.0,"[таким, образом, никто, внес, большего, вклада...","[[-0.08076643198728561, -0.03717906400561333, ..."
1,2,"Можно с уверенностью сказать, что все полезное...",3.0,"[уверенностью, сказать, полезное, хорошее, сов...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2,3,Славяне должны на нас работать. Если они нам б...,3.0,"[славяне, должны, работать, нам, нужны, могут,...","[[-0.04596193755666415, -0.04917717600862185, ..."
3,4,"Русское государство росло, развивалось из свои...",3.0,"[русское, государство, росло, развивалось, сво...","[[-0.1227731704711914, -0.11972755752503872, -..."
4,5,Поставим власть под контроль народа,3.0,"[поставим, власть, контроль, народа]","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
...,...,...,...,...,...
304,305,Социальное. Соответствующий подход предполагае...,0.0,"[социальное, соответствующий, подход, предпола...","[[-0.09335160627961159, -0.05264603905379772, ..."
305,306,Рост продуктивности человеческой деятельности ...,0.0,"[рост, продуктивности, человеческой, деятельно...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
306,307,"Социальная дифференциация, в том числе по этни...",0.0,"[социальная, дифференциация, числе, этническом...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
307,308,Демографический рост и расширение экономическо...,0.0,"[демографический, рост, расширение, экономичес...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."


In [153]:
np.shape(df['medVec'][0])

(300,)

In [124]:

w2v_model = Word2Vec(
   min_count=20,
                     window=4,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     )

#BUILD_VOCAB()
t = time.time()
w2v_model.build_vocab(data, progress_per=100)
print("Time to build vocab': {} mins".format(round((time.time() - t) / 60, 2)))

#TRAIN()
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))

Time to build vocab': 0.0 mins
Time to train the model: 0.0 mins


In [125]:
w2v_model.init_sims(replace=True)
w2v_model.wv.save_word2vec_format(r'C:\Users\Zero\Desktop\models\hachitini.bin', binary=True)

In [126]:
model1 = w2v_model.wv.load_word2vec_format(r'models\hachi.bin', binary=True)
model1

In [127]:
model1.most_similar(positive=["хачи"])

[('чурок', 0.9904347658157349),
 ('хачам', 0.9889359474182129),
 ('убивайте', 0.9868521690368652),
 ('россии', 0.9765247106552124),
 ('будут', 0.9525323510169983),
 ('убей', 0.8711485862731934),
 ('снова', 0.78118896484375),
 ('нам', 0.7252949476242065),
 ('своей', 0.7228883504867554),
 ('днем', 0.7154790759086609)]

In [128]:
import numpy as np

index2word_set = set(model1.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

<ipython-input-128-f9aa2771d6b9>:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(model1.wv.index2word)


In [129]:

from pathlib import Path
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras import optimizers


# build a model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=10000))
model.add(Dropout(0.25))
model.add(Dense(50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
optimiz = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy',optimizer = optimiz ,metrics = ['accuracy'])

# convert texts to TF-IDF matrices


 fit the model
hist = model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 5,batch_size = 64)

IndentationError: unexpected indent (<ipython-input-129-517af12cf84f>, line 21)

In [42]:
listok=["я убиваю пидоров"]
deist='убить'
score=[]
slovar={}
for i in range(len(listok)):
    s1_afv = avg_feature_vector(deist, model=model1, num_features=300, index2word_set=index2word_set)
    s2_afv = avg_feature_vector(listok[i], model=model1, num_features=300, index2word_set=index2word_set)
    sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
    score.append(sim)
    slovar.update({deist+' / '+listok[i]:sim})
sorted_tuples = sorted(slovar.items(), key=lambda item: item[1])

sorted_dict = {k: v for k, v in sorted_tuples}

print(sorted_dict)  # {1: 1, 3: 4, 2: 9}

{'убить / я убиваю пидоров': nan}


In [83]:
df = df.astype({'class': np.int})
df

,id,text,class,tokenized,medVec
0,1,Таким образом никто не внес большего вклад...,3,"[таким, образом, никто, внес, большего, вклада...","[-0.040627204, -0.015365396, -0.047234748, -0...."
1,2,"Можно с уверенностью сказать, что все полезное...",3,"[уверенностью, сказать, полезное, хорошее, сов...","[-0.039338265, -0.028398223, -0.0673505, -0.01..."
2,3,Славяне должны на нас работать. Если они нам б...,3,"[славяне, должны, работать, нам, нужны, могут,...","[-0.036592077, -0.06395722, -0.045187872, -0.0..."
3,4,"Русское государство росло, развивалось из свои...",3,"[русское, государство, росло, развивалось, сво...","[-0.01085669, -0.022112658, -0.07141073, -0.03..."
4,5,Поставим власть под контроль народа,3,"[поставим, власть, контроль, народа]","[-0.052434314, -0.07669518, -0.019612849, -0.0..."
...,...,...,...,...,...
304,305,Социальное. Соответствующий подход предполагае...,0,"[социальное, соответствующий, подход, предпола...","[-0.03102859, -0.050743006, -0.037808374, -0.0..."
305,306,Рост продуктивности человеческой деятельности ...,0,"[рост, продуктивности, человеческой, деятельно...","[-0.022473654, -0.052780617, -0.06827721, -0.0..."
306,307,"Социальная дифференциация, в том числе по этни...",0,"[социальная, дифференциация, числе, этническом...","[-0.017352613, -0.019620007, -0.04677039, -0.0..."
307,308,Демографический рост и расширение экономическо...,0,"[демографический, рост, расширение, экономичес...","[-0.03736097, -0.06539034, -0.062378306, -0.05..."


In [130]:
y = df['class']
X = df["tokenized"]

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [131]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

vocab size : 19


In [132]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [135]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_train)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)

print ('shape for training set : ',train_vecs_w2v.shape,
      '\nshape for test set : ', test_vecs_w2v.shape)

shape for training set :  (247, 300) 
shape for test set :  (62, 300)


<ipython-input-132-de3fa4e5f276>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec += w2v_model[word].reshape((1, size)) * tfidf[word]
<ipython-input-132-de3fa4e5f276>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec += w2v_model[word].reshape((1, size)) * tfidf[word]


In [139]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=300))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_97 (Dense)             (None, 128)               38528     
_________________________________________________________________
dropout_34 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_98 (Dense)             (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [140]:
history = model.fit(train_vecs_w2v, y_train, epochs=20, batch_size=50,
                   validation_data=(test_vecs_w2v,y_test))
loss, accuracy = model.evaluate(train_vecs_w2v, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_vecs_w2v, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/20
5/5 [==============================] - 1s 75ms/step - loss: 0.9865 - accuracy: 0.3450 - val_loss: 0.6168 - val_accuracy: 0.3548
Epoch 2/20
5/5 [==============================] - 0s 17ms/step - loss: 0.8589 - accuracy: 0.3492 - val_loss: 0.6151 - val_accuracy: 0.3548
Epoch 3/20
5/5 [==============================] - 0s 17ms/step - loss: 0.9260 - accuracy: 0.2996 - val_loss: 0.6133 - val_accuracy: 0.3548
Epoch 4/20
5/5 [==============================] - 0s 13ms/step - loss: 0.8363 - accuracy: 0.3641 - val_loss: 0.6119 - val_accuracy: 0.3548
Epoch 5/20
5/5 [==============================] - 0s 16ms/step - loss: 1.0289 - accuracy: 0.3365 - val_loss: 0.6104 - val_accuracy: 0.3548
Epoch 6/20
5/5 [==============================] - 0s 19ms/step - loss: 0.8002 - accuracy: 0.3057 - val_loss: 0.6089 - val_accuracy: 0.3548
Epoch 7/20
5/5 [==============================] - 0s 19ms/step - loss: 0.5575 - accuracy: 0.3031 - val_loss: 0.6073 - val_accuracy: 0.3548
Epoch 8/20
5/5 [===========

In [144]:
t = Tokenizer(num_words=300)
texts = ['что за херня', 'что проихсодит', 'убейте меня нахуй']
t.fit_on_texts(texts)
X = t.texts_to_matrix(texts, mode='tfidf')

model.predict(X)


array([[0.5145784 ],
       [0.48452798],
       [0.4570892 ]], dtype=float32)

In [141]:
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
plot_history(history)

KeyError: 'acc'

In [51]:

from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras import optimizers

In [80]:

import tensorflow as tf
from pathlib import Path
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras import optimizers



from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
 df['medVec'], df['class'], random_state=0)


# build a model
model = Sequential()
model.add(Dense(300, activation='relu', input_dim=10000))
model.add(Dropout(0.25))
model.add(Dense(50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
optimiz = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy',optimizer = optimiz ,metrics = ['accuracy'])

# convert texts to TF-IDF matrices


hist = model.fit(X_train1,y_train1,validation_data = (X_test1,y_test1),epochs = 5,batch_size = 64)




NameError: name 'X_train1' is not defined

In [81]:
from pathlib import Path
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras import optimizers


# read data
X_train, X_test, y_train, y_test = train_test_split(
 df['medVec'], df['class'], random_state=0)



# build a model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=100))
model.add(Dropout(0.25))
model.add(Dense(50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
optimiz = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy',optimizer = optimiz ,metrics = ['accuracy'])


# fit the model
hist = model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 10,batch_size = 64)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [73]:
# predict our own texts
texts = ['я ебу хачей', 'чтоб вы сдохли', 'все ок']
t.fit_on_texts(texts)
X = t.texts_to_matrix(texts, mode='tfidf')

model.predict(X)

array([[0.9543379 ],
       [0.99722654],
       [0.9757453 ]], dtype=float32)

In [171]:
df['class'] = df['class'].astype('int32')

In [167]:
df['tokenized']

0      [таким, образом, никто, внес, большего, вклада...
1      [уверенностью, сказать, полезное, хорошее, сов...
2      [славяне, должны, работать, нам, нужны, могут,...
3      [русское, государство, росло, развивалось, сво...
4                   [поставим, власть, контроль, народа]
                             ...                        
304    [социальное, соответствующий, подход, предпола...
305    [рост, продуктивности, человеческой, деятельно...
306    [социальная, дифференциация, числе, этническом...
307    [демографический, рост, расширение, экономичес...
308    [соответственно, политика, возникает, связи, у...
Name: tokenized, Length: 309, dtype: object

In [183]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df['text'], df['class'], test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [184]:
y_train

276    0
29     3
247    0
258    0
149    1
      ..
62     3
240    0
143    1
121    1
129    1
Name: class, Length: 222, dtype: int32

In [185]:
from sklearn.pipeline import Pipeline
# pipeline позволяет объединить в один блок трансформер и модель, что упрощает написание кода и улучшает его читаемость
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer преобразует тексты в числовые вектора, отражающие важность использования каждого слова из некоторого набора слов (количество слов набора определяет размерность вектора) в каждом тексте
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
# линейный классификатор и классификатор методом ближайших соседей
from sklearn import metrics
# набор метрик для оценки качества модели
from sklearn.model_selection import GridSearchCV
# модуль поиска по сетке параметров

In [186]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
sgd_ppl_clf.fit(X_train, y_train)
knb_ppl_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

In [214]:
pred = knb_ppl_clf.predict(['хахаха смер.ть хачам'])
print(pred)

[0]
